In [18]:
import warnings
warnings.filterwarnings(action='ignore')

import os
import gc
import cv2
import random
import argparse
import numpy as np
import pandas as pd
from glob import glob

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from tqdm.autonotebook import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

import wandb

from utils_.set_path import *
from utils_.loss import FocalLoss, LabelSmoothingLoss, F1Loss
from utils_.get_class_weight import calc_class_weight
from runner.pytorch_timm import TimmModel
import matplotlib.pyplot as plt
# from runner.train_runner import CustomTrainer
# from data.dataset import CustomTrainDataset

### Preparation

In [19]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [20]:
seed_everything(909)

In [5]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

def get_transforms(img_size, transforms):
    if transforms:
         return A.Compose([
                    # A.CenterCrop(height=384, width=384, p=1.0),
                    A.Resize(img_size[0], img_size[1]),
                    A.HorizontalFlip(p=0.5),
                    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0),
                    ToTensorV2()
                    ])
    else:
        return A.Compose([
                    # A.CenterCrop(height=384, width=384, p=1.0),
                    A.Resize(img_size[0], img_size[1]),
                    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0),
                    ToTensorV2()
                    ])

class CustomTrainDataset(Dataset):
    _mask_labels = {"mask1": 0, "mask2": 0, "mask3": 0, "mask4": 0, "mask5": 0, "incorrect_mask": 1, "normal": 2}
    _gender_labels = {"male": 0, "female": 1}

    def __init__(self, model_name, data_dir, folder_list, resize, transforms):
        self.model_name = model_name
        self.data_dir = data_dir
        self.folder_list = folder_list
        self.resize = resize
        self.transforms = transforms
        
        self.image_paths = []
        self.mask_labels, self.gender_labels, self.age_labels = [], [], []
        self.setup()
        

    def setup(self):
        for folder in self.folder_list:
            img_folder = os.path.join(self.data_dir, folder) # ('/workspace/data/train/image', 000004_male_Asian_54)
            for file_name in os.listdir(img_folder): # ('mask1.jpg', 'mask2.jpg', ... )
                _file_name, _ = os.path.splitext(file_name) # ('mask1', '.jpg')
                img_path = os.path.join(self.data_dir, folder, file_name)  # (resized_data, 000004_male_Asian_54, mask1.jpg)
                
                _, gender, _, age = folder.split("_")
                # mask_label = self._mask_labels[_file_name]                
                gender_label = self._gender_labels[gender]
                age = int(age)
                age_label = 0 if age < 30 else 1 if age < 60 else 2
                
                self.image_paths.append(img_path)
                if self.model_name == 'Mask': self.mask_labels.append(0)
                elif self.model_name == 'Gender': self.gender_labels.append(gender_label)
                elif self.model_name == 'Age': 
                    self.age_labels.append(age_label)
    def __getitem__(self, index):
        assert self.transforms is not None, "transform 을 주입해주세요"
        
        image = cv2.imread(self.image_paths[index])
        trfm = get_transforms(self.resize, self.transforms)
        image = trfm(image=image)['image']
        
        if self.model_name == 'Mask':
            mask_label = self.mask_labels[index]
            return image, mask_label
        elif self.model_name == 'Gender':
            gender_label = self.gender_labels[index]
            return image, gender_label
        elif self.model_name == 'Age':
            age_label = self.age_labels[index]
            return image, age_label
        
    def __len__(self):
        return len(self.image_paths)

class CustomTestDataset(Dataset):
    def __init__(self, img_paths, resize, transforms=None):
        self.img_paths = img_paths
        self.resize = resize
        self.transforms = transforms

    def __getitem__(self, index):
        image = cv2.imread(self.img_paths[index])
        trfm = get_transforms(self.resize, self.transforms)
        image = trfm(image=image)['image']
            
        return image

    def __len__(self):
        return len(self.img_paths)

In [6]:
model_name = 'Age'
image_resize = [512, 384]
gc.collect() # python 자원 관리 
torch.cuda.empty_cache() # gpu 자원관리

data_dir = TRAIN_IMG_FOLDER_PATH
model_dir = MODEL_SAVE_PATH
train_df = pd.read_csv(TRAIN_CSV_PATH)
miss_label = ['006424', '006339', '003713', '003437', '003421', '003399', '003294', '003169', '003113', '003014', '001520', '001266', '000725', '000647']
drop_index = []
for ml in miss_label:
    drop_index.append(train_df[train_df['id']==ml].index[0])
for index in drop_index:
    train_df = train_df.drop(index)
train_, val_ = train_test_split(train_df, test_size=0.2, random_state=909)

train_dataset = CustomTrainDataset(model_name, data_dir, train_['path'].values, image_resize, transforms=True)
val_dataset = CustomTrainDataset(model_name, data_dir, val_['path'].values, image_resize, transforms=False)
train_dataloader = DataLoader(train_dataset, batch_size = 32, shuffle=True, num_workers=8)
valid_dataloader = DataLoader(val_dataset, batch_size = 32, shuffle=False, num_workers=8)


In [7]:
# python train.py --seed 909 --epochs 100 --resize 256 192 --batch_size 32 --criterion CrossEntropy --model resnet34 --lr 1e-6

In [8]:
import torch.nn as nn
import torchvision.models as models
import timm

class TimmModel(nn.Module):
    def __init__(self, model, num_classes, pretrained=True):
        super().__init__()
        self.num_classes = num_classes
        self.pretrained = pretrained
        
        self.model = timm.create_model(model, pretrained=self.pretrained)
        self.fc = nn.Sequential(nn.Dropout(p=0.2, inplace=True),
                               nn.Linear(1000, 512),
                               nn.Dropout(p=0.2, inplace=True),
                               nn.Linear(512, self.num_classes),
                               )

    def forward(self, x):
        x = self.model(x)
        x = self.fc(x)
        return x

In [9]:
device = torch.device('cuda')
model = TimmModel('resnet34', num_classes=3, pretrained=True).to(device)
model.load_state_dict(torch.load("/opt/ml/geunuk/model/Project54/Age_[resnet34]_[score0.8015]_[loss0.3479].pt"))
# model = nn.DataParallel(model)


<All keys matched successfully>

In [10]:
class CustomTrainer():
    def __init__(self, args = None, model = None, valid_dataloader = None, optimizer = None, scheduler = None, criterion = None, device = None):
        self.args = args
        self.model = model
        self.valid_dataloader = valid_dataloader
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.criterion = criterion
        self.device = device 

    def validation(self):
        self.model.eval()
        val_loss = []
        preds, trues = [], []
        print('='*25, f'VALID', '='*25)
        with torch.no_grad():
            for imgs, labels in tqdm(iter(self.valid_dataloader), bar_format='{l_bar}{bar:50}{r_bar}{bar:-10b}'):
                imgs = imgs.float().to(self.device)
                labels = labels.to(self.device)

                logit = self.model(imgs)

                loss = self.criterion(logit, labels)

                val_loss.append(loss.item())

                preds += logit.argmax(1).detach().cpu().numpy().tolist()
                trues += labels.detach().cpu().numpy().tolist()

            _val_loss = np.mean(val_loss)
            _val_acc = f1_score(trues, preds, average='macro')
        return _val_loss, _val_acc

### Validation f1 score 확인

In [11]:
#class_weight = calc_class_weight(train_dataset, model_name, 3, device)
#criterion = FocalLoss(weight=class_weight).to(device)
criterion = nn.CrossEntropyLoss().to(device)
# optimizer = torch.optim.AdamW(model.parameters(), lr=args.lr, betas=(0.9, 0.999), eps=1e-8, weight_decay=1e-3)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, threshold_mode='abs',min_lr=1e-9, verbose=True)


In [12]:
trainer = CustomTrainer(model = model, valid_dataloader = valid_dataloader, criterion=criterion, device=device)

In [13]:
targets_img = None
preds, targets = torch.Tensor().to(device), torch.Tensor().to(device)
preds_all, targets_all = torch.Tensor().to(torch.int32).to(device), torch.Tensor().to(torch.int32).to(device)
with torch.no_grad():
    for X, y in tqdm(iter(valid_dataloader)):
        model.eval()
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        pred = pred.argmax(dim=1)
        indices = (pred != y).nonzero().squeeze()
        preds_all = torch.cat((preds_all, pred))
        targets_all = torch.cat((targets_all, y))
        if indices.nelement() != 0:
            target_img = X[indices]
            if target_img.dim() == 3:
                target_img = target_img.unsqueeze(dim=0)
            
            if targets_img is None:
                targets_img = target_img
            else:
                targets_img = torch.cat((targets_img, target_img), dim=0)
            
            ps = pred[indices]
            ys = y[indices]
            if ys.dim() == 0:
                ys = ys.unsqueeze(dim=0)
                ps = ps.unsqueeze(dim=0)
            preds = torch.cat((preds, ps))
            targets = torch.cat((targets, ys))

In [14]:
len(val_dataset)

3766

In [15]:
from sklearn.metrics import f1_score

In [17]:
f1_score(targets_all.cpu(), preds_all.cpu(), average="macro")

0.8134464235227594

### Class activation map 출력

In [132]:
visualization = {}
def hook(module, input, output):
    if module == model.model.layer4:
        visualization["test"] = output
        
for module in model.modules():
    module.register_forward_hook(hook)  

In [133]:
def denorm_image(img, size):
    c1 = torch.zeros(1, *size) + 0.229
    c2 = torch.zeros(1, *size) + 0.224
    c3 = torch.zeros(1, *size) + 0.225
    std_cn = torch.cat((c1, c2, c3), dim=0)

    c1 = torch.zeros(1, *size) + 0.485
    c2 = torch.zeros(1, *size) + 0.456
    c3 = torch.zeros(1, *size) + 0.406
    std_mean = torch.cat((c1, c2, c3), dim=0)
    
    return img.cpu() * std_cn + std_mean

In [134]:
val_iter = iter(valid_dataloader)

In [135]:
sample = next(val_iter)

In [ ]:
warnings.filterwarnings(action="ignore")
plt.figure(figsize=(15, 25))
with torch.no_grad():
    for i in range(32):
        model.eval()
        image_idx=i

        label = sample[1][image_idx]
        sample_image = sample[0][image_idx].unsqueeze(0)

        output = model(sample_image.cuda())
        output = output.argmax(dim=-1)
        

        activation_map = list(visualization.values())
        activation_map = activation_map[0]

        class_weight = model.fc[3].weight[int(output)].unsqueeze(-1).unsqueeze(-1)

        cam_ = torch.squeeze(activation_map * class_weight)
        cam = torch.sum(cam_, axis=0)
        cam = cam.detach().cpu().numpy()

        final_cam = cv2.resize(cam, (384, 412), interpolation=cv2.INTER_CUBIC)
        plt.subplot(7, 5, i+1)
        plt.axis("off")
        plt.title(f't:{label.item()}, p:{output.item()}')
        plt.imshow(denorm_image(sample_image[0], sample_image[0].shape[1:]).permute(1, 2, 0).flip(2))
        plt.imshow(final_cam, alpha=0.5, cmap="jet")

### Validation set 잘못된 prediction 확인

In [150]:

misprediction_count = {
    (0, 1) : 0,
    (0, 2) : 0,
    (1, 0) : 0,
    (1, 2) : 0,
    (2, 0) : 0,
    (2, 1) : 0
}
misprediction_tostring = {
    (0, 1) : "pred <30, label 30to60",
    (0, 2) : "pred <30, label >60",
    (1, 0) : "pred 30to60, label <30",
    (1, 2) : "pred 30to60, label >60",
    (2, 0) : "pred >60, label <30",
    (2, 1) : "pred >60, label 30to60",
}

In [61]:
for pred, label in tqdm(zip(preds, targets)):
    misprediction_count[(pred.item(), label.item())] += 1

In [62]:
test = np.array(list(zip(preds.to(torch.uint8).cpu(), targets.to(torch.uint8).cpu())))
indices = np.where((test[:, 0] == 0) & (test[:, 1] == 1))
indices = indices[0]

In [63]:
misprediction_count

{(0, 1): 68, (0, 2): 0, (1, 0): 45, (1, 2): 120, (2, 0): 0, (2, 1): 108}

In [64]:
for key, value in misprediction_count.items():
    print(f'{misprediction_tostring[key]}: {value}')

pred <30, label 30to60: 68
pred <30, label >60: 0
pred 30to60, label <30: 45
pred 30to60, label >60: 120
pred >60, label <30: 0
pred >60, label 30to60: 108


### Validation set 잘못된 prediction 이미지 확인

In [19]:
c1 = torch.zeros(1, 256, 192) + 0.229
c2 = torch.zeros(1, 256, 192) + 0.224
c3 = torch.zeros(1, 256, 192) + 0.225
std_cn = torch.cat((c1, c2, c3), dim=0)

c1 = torch.zeros(1, 256, 192) + 0.485
c2 = torch.zeros(1, 256, 192) + 0.456
c3 = torch.zeros(1, 256, 192) + 0.406
std_mean = torch.cat((c1, c2, c3), dim=0)

In [20]:
import matplotlib.pyplot as plt

In [21]:
# class_dict = {0: "male", 1:"female"}
class_dict = {0: "<30", 1:"30to60", 2:">60"}

In [42]:
class_counts_train = {i:0 for i in range(3)}
class_counts_val = {i:0 for i in range(3)}

In [40]:
for img, target in val_dataset:
    class_counts_val[target]+=1

In [46]:
class_counts_train

{0: 7280, 1: 6839, 2: 1001}

In [41]:
class_counts_val

{0: 1687, 1: 1750, 2: 343}

In [45]:
for img, target in train_dataset:
    class_counts_train[target]+=1

In [ ]:
test = np.array(list(zip(preds.to(torch.uint8).cpu(), targets.to(torch.uint8).cpu())))
indices = np.where((test[:, 0] == 0) & (test[:, 1] == 1))
indices = indices[0]

plt.figure(figsize=(30, 90))
for idx, (img, pred, targ) in enumerate(zip(targets_img[indices], preds[indices], targets[indices])):
    plt.axis("off")
    plt.subplot(20, 10, idx+1)
    plt.title(f"pred:{class_dict[pred.item()]}, label:{class_dict[targ.item()]}", color="white")
    plt.imshow((img.cpu() * std_cn + std_mean).permute(1, 2, 0).flip(2))

In [ ]:
test = np.array(list(zip(preds.to(torch.uint8).cpu(), targets.to(torch.uint8).cpu())))
indices = np.where((test[:, 0] == 0) & (test[:, 1] == 2))
indices = indices[0]

plt.figure(figsize=(30, 90))
for idx, (img, pred, targ) in enumerate(zip(targets_img[indices], preds[indices], targets[indices])):
    plt.axis("off")
    plt.subplot(20, 10, idx+1)
    plt.title(f"pred:{class_dict[pred.item()]}, label:{class_dict[targ.item()]}",color="white")
    plt.imshow((img.cpu() * std_cn + std_mean).permute(1, 2, 0).flip(2))

In [ ]:
test = np.array(list(zip(preds.to(torch.uint8).cpu(), targets.to(torch.uint8).cpu())))
indices = np.where((test[:, 0] == 1) & (test[:, 1] == 0))
indices = indices[0]

plt.figure(figsize=(30, 90))
for idx, (img, pred, targ) in enumerate(zip(targets_img[indices], preds[indices], targets[indices])):
    plt.axis("off")
    plt.subplot(20, 10, idx+1)
    plt.title(f"pred:{class_dict[pred.item()]}, label:{class_dict[targ.item()]}", color="white")
    plt.imshow((img.cpu() * std_cn + std_mean).permute(1, 2, 0).flip(2))

In [ ]:
test = np.array(list(zip(preds.to(torch.uint8).cpu(), targets.to(torch.uint8).cpu())))
indices = np.where((test[:, 0] == 1) & (test[:, 1] == 2))
indices = indices[0]

plt.figure(figsize=(30, 90))
for idx, (img, pred, targ) in enumerate(zip(targets_img[indices], preds[indices], targets[indices])):
    plt.axis("off")
    plt.subplot(20, 10, idx+1)
    plt.title(f"pred:{class_dict[pred.item()]}, label:{class_dict[targ.item()]}", color="white")
    plt.imshow((img.cpu() * std_cn + std_mean).permute(1, 2, 0).flip(2))

In [ ]:
test = np.array(list(zip(preds.to(torch.uint8).cpu(), targets.to(torch.uint8).cpu())))
indices = np.where((test[:, 0] == 2) & (test[:, 1] == 0))
indices = indices[0]

plt.figure(figsize=(30, 90))
for idx, (img, pred, targ) in enumerate(zip(targets_img[indices], preds[indices], targets[indices])):
    plt.axis("off")
    plt.subplot(20, 10, idx+1)
    plt.title(f"pred:{class_dict[pred.item()]}, label:{class_dict[targ.item()]}", color="white")
    plt.imshow((img.cpu() * std_cn + std_mean).permute(1, 2, 0).flip(2))

In [ ]:
test = np.array(list(zip(preds.to(torch.uint8).cpu(), targets.to(torch.uint8).cpu())))
indices = np.where((test[:, 0] == 2) & (test[:, 1] == 1))
indices = indices[0]

plt.figure(figsize=(30, 90))
for idx, (img, pred, targ) in enumerate(zip(targets_img[indices], preds[indices], targets[indices])):
    plt.axis("off")
    plt.subplot(20, 10, idx+1)
    plt.title(f"pred:{class_dict[pred.item()]}, label:{class_dict[targ.item()]}", color="white")
    plt.imshow((img.cpu() * std_cn + std_mean).permute(1, 2, 0).flip(2))